In [1]:
#This code mostly functions to run the registration and transformation on the TLC to FRC images and airways. There are two blocks of code that are not funcitoning but these functions are used to retrieve files 
#so this work can be done manually. The issue with why the code isn't functioning on all the cases is due to the parameter files being off so this would need to be fixed before moving forward on all 80 cases. 
#The other option if the parameters can't be resolved would be to change the registration process to AntsPy instead of elastix. This should be a pretty quick change and should only affect the registrations and
#transformation functions in the code. 


#Code for atuomatic registration and saving files to U-AllysonRegistration File

import os
import shutil
import re
import zipfile
import subprocess
import dicom2nifti
import SimpleITK as sitk
import numpy as np
import nibabel as nib



# Function creates Case Folder in U-AllysonRegistration

def create_case_folder(case_number):
    if case_number < 10:
        folder_name = f"Case-0{case_number}"
    else:
        folder_name = f"Case-{case_number}"

    destination_folder = "z:/U-AllysonRegistration"
    folder_path = os.path.join(destination_folder, folder_name)

    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    return folder_path


#Not Functional with certain cases: Function finds and copies the FRC file to U-AllysonRegistration and Case ID

def load_FRC_find_CaseID(case_number, folder_path):
    found_file = None
    case_id = None
    destination_folder = folder_path
    source_folder = "z:/D-Images/SPIROMICS-SubStudy/2-Results-CheckedDoneTemp/1-Done"

    for root, dirs, files in os.walk(source_folder):
        for dir_name in dirs:
            if dir_name.startswith(f"Case-{case_number}"):
                case_folder = os.path.join(root, dir_name)
                additional_images_folder = os.path.join(case_folder, "Additional Images")

                if os.path.exists(additional_images_folder):
                    for root2, dirs2, files2 in os.walk(additional_images_folder):
                        for file in files2:
                            if "0.5" in file and "150KV" in file:
                                found_file = os.path.join(root2, file)
                                print(found_file)
                                destination_path = os.path.join(destination_folder, os.path.basename(found_file))
                                shutil.copy(found_file, destination_path)

                                # Extract the Case ID from the file name
                                case_id_match = re.search(r'(?<=SPI-)([^_]+)-PBV_SPIROMICS', found_file)
                                case_id = case_id_match.group(1)
                                print(f"Case ID: {case_id}")
                                break
    return case_id


#Not Working: Function finds the TLC folder in teamwork and copies to Case Folder

def load_TLC_folder(case_id, folder_path):
    found_file = None
    source_folder = r"\\lc-rs-store25.hpc.uiowa.edu\teamwork\public_resources\Scan_Exchange\for_Junfeng\SPIR--ATS-VessDys-TLCs\1-UseThis-Unzipped-APKs"
    destination_folder = folder_path

    for root, dirs, files in os.walk(source_folder):
        for dir in dirs:
            if case_id in dir:
                found_file = os.path.join(root, dir) 

                destination_apk_path = os.path.join(destination_folder, found_file)
                shutil.copy(found_file, destination_apk_path)
    print("TLC folder copied to case file successfully")


#Function finds the TLC DICOM folder, loads the DICOM, and saves this as a .nii.gz file: Run time 1m 10s

def load_TLC_DICOM(case_id, folder_path):
    source_folder = folder_path

    subfolders = [f for f in os.listdir(source_folder) if os.path.isdir(os.path.join(source_folder, f))]
    target_subfolder = None

    for subfolder in subfolders:
        if "TLC" in subfolder and str(case_id) in subfolder:
            target_subfolder = subfolder
            break

    if target_subfolder is None:
        print(f"Error: Subfolder with case id and 'TLC' not found.")
        return None

    target_subfolder_path = os.path.join(source_folder, target_subfolder)

    dicom_folder_path = None
    for root, dirs, files in os.walk(target_subfolder_path):
        if 'dicom' in dirs:
            dicom_folder_path = os.path.join(root, 'dicom')
            print(dicom_folder_path)
            break

    nifti_output_path = source_folder

    dicom2nifti.convert_directory(dicom_folder_path, nifti_output_path, compression=True, reorient=True)
    print('TLC DICOM loaded successfully')


#Function renames the TLC and FRC files for easy recognition and use

def rename_files(case_id, folder_path):
    source_folder = folder_path

    for filename in os.listdir(source_folder):
        # Check if the file name contains a certain case id
        if "SN150KV" in filename and case_id in filename and filename.endswith(".nii.gz"):
            new_filename = "FRC-DICOM.nii.gz"
        # Check if the file name contains "inspiration" and ends with .nii.gz
        elif "inspiration" in filename and filename.endswith(".nii.gz"):
            new_filename = "TLC-DICOM.nii.gz"
        else:
            # Skip files that don't match the specified conditions
            continue

        old_file_path = os.path.join(source_folder, filename)
        new_file_path = os.path.join(source_folder, new_filename)
        
        # Rename the file
        os.rename(old_file_path, new_file_path)
    print('Files renamed successfully')


#Function performs Registration: Run time 3m 30s

def run_registration(folder_path):
    source_folder = folder_path

    # Path to Elastix executable
    elastix_path = "C:/Users/akuhn1/Downloads/elastix-5.1.0-win64/elastix.exe" 

    # Path to the parameter file
    parameter_file1 = "z:/U-AllysonRegistration/Parameters.Par0008.affine.txt"
    # parameter_file2 = "z:/U-AllysonRegistration/Par0004.bs_base.NRP08.All.txt" 
    # parameter_file3 = "z:/U-AllysonRegistration/Par0004.rigid(1).txt" 

    # Path to fixed and moving images
    fixed_image = os.path.join(source_folder, "FRC-DICOM.nii.gz")
    moving_image = os.path.join(source_folder, "TLC-DICOM.nii.gz")

    # Path to output folder
    output_folder_path = os.path.join(source_folder, "Registration_Outputs")
    os.makedirs(output_folder_path, exist_ok=True)

    # Output directory for the registered image
    output_directory = os.path.join(source_folder, output_folder_path)

    # Run Elastix registration
    registration_command = [elastix_path, "-f", fixed_image, "-m", moving_image, "-out", output_directory, "-p", parameter_file1]#, "-p", parameter_file3]
    subprocess.run(registration_command)
    print("Registration complete")


#Function recenters the TLC airway

def recentering(folder_path):
    source_folder = folder_path

    SEGMENTS = ['ZUNU_vida-airtree.hdr']
    ImgFilename = os.path.join(source_folder, "TLC-DICOM.nii.gz")
    DestinationFolder = source_folder

    for root, dirs, files in os.walk(source_folder):
        # Check if "ABCD" file exists in the current folder
        if "ZUNU_vida-airtree.hdr" in files:
            # Save the folder containing "ZUNU_vida-airtree.hdr" as "APK_Folder"
            ApkFolder = os.path.abspath(root)

    ImgFilename = '/'.join(ImgFilename.split('\\'))
    ApkFolder = '/'.join(ApkFolder.split('\\'))
    DestinationFolder = '/'.join(DestinationFolder.split('\\'))

    iim = sitk.ReadImage(ImgFilename)

    for SEG in SEGMENTS:
        segfn = os.path.join(ApkFolder, SEG)
        segim = sitk.ReadImage(segfn)
        snda = sitk.GetArrayFromImage(segim)
        snds = np.flip(snda, axis=0)
        sndb = np.flip(snds, axis=1)
        fseg = sitk.GetImageFromArray(sndb)
        fseg.CopyInformation(iim)
        outfn = SEG[0:-4] + '-recentered.nii.gz'
        sitk.WriteImage(fseg, os.path.join(DestinationFolder, outfn))
        print('saved:', outfn)

    print('Recentering process complete')


#Function performs Transformation: Run time 40s

def run_transformation(folder_path):
    source_folder = folder_path

    # Initialize variables to store file paths
    transform_parameters1 = None
    transform_parameters2 = None

    # Iterate through the source folder
    for root, dirs, files in os.walk(source_folder):
        # Check if "Registration Outputs" folder is present
        if "Registration_Outputs" in dirs:
            registration_outputs_folder = os.path.join(root, "Registration_Outputs")

            # Iterate through files in "Registration Outputs" folder
            for filename in os.listdir(registration_outputs_folder):
                file_path = os.path.join(registration_outputs_folder, filename)

                # Check for "TransformParameters.0.txt" and "TransformParameters.1.txt"
                if filename == "TransformParameters.0.txt":
                    transform_parameters1 = file_path
                elif filename == "TransformParameters.1.txt":
                    transform_parameters2 = file_path

    # Path to Transformix executable
    transformix_path = "C:/Users/akuhn1/Downloads/elastix-5.1.0-win64/transformix.exe"

    # Path to input image
    input_image = os.path.join(source_folder, "ZUNU_vida-airtree-recentered.nii.gz")

    # Path to output folder
    output_folder_path = os.path.join(source_folder, "Transformation_Outputs")
    os.makedirs(output_folder_path, exist_ok=True)

    # Run Transformix transform
    transformation_command = [transformix_path, "-in", input_image, "-out", output_folder_path, "-tp", transform_parameters2, "-tp", transform_parameters1]
    subprocess.run(transformation_command)
    print("Transformation completed successfully")
    

#Function combines transformation reult segments at a given threshold 
def apply_threshold(folder_path):
    source_folder = folder_path

    # Load NIfTI file
    file_path = os.path.join(source_folder, "Transformation_Outputs/result.nii")
    img = nib.load(file_path)

    # Get the data array
    data = img.get_fdata()

    threshold_value = 127.5  # Adjust this value based on your data
    combined_segmentation = np.where(data > threshold_value, 255, 0)

    # Save the result to a new NIfTI file
    combined_img = nib.Nifti1Image(combined_segmentation, img.affine)
    modified_file_path = os.path.join(source_folder, "Transformed_Airway.nii.gz")
    nib.save(combined_img, modified_file_path)

    print(f"Modified file saved at: {modified_file_path}")
           

    





In [ ]:
case_number = 22

folder_path = create_case_folder(case_number)

print(folder_path)

# case_id = load_FRC_find_CaseID(case_number, folder_path)

case_id = "JH110088"

# load_TLC_folder(case_id, folder_path)

load_TLC_DICOM(case_id, folder_path)

rename_files(case_id, folder_path)

run_registration(folder_path)

recentering(folder_path)

run_transformation(folder_path)

apply_threshold(folder_path)